In [1]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_tx_task import SingleChTxTask
from rfsoc_rfdc.receiver.single_ch_rx_task import SingleChRxTask

from rfsoc_rfdc.transmitter.multi_ch_tx_task import MultiChTxTask
from rfsoc_rfdc.receiver.multi_ch_rx_task import MultiChRxTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask
from rfsoc_rfdc.rfdc_config import ZCU216_CONFIG

import sys
import os
import time

In [2]:
# RFSoC experiment configuration
# +------------------------------------------------------------------------------------------------+
# | Feature: Single Channel 312.5 MHz BW Tx/Rx Experiment                                          |
# +-----------------+--------------------+----------------+-----------+---------------+------------+
# | DAC Sample Rate | Interpolation Rate | IQ Sample Rate | Bandwidth | NCO           | Ref. Clock |
# | 4.0 GSPS        | 40x                | 100 MSPS       | 100M      | 1000:1800:200M| 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+---------------+------------+
# | ADC Sample Rate | Decimation Rate    | IQ Sample Rate | Bandwidth | NCO           | Ref. Clock |
# | 2.0 GSPS        | 20x                | 100 MSPS       | 100M      | 1000:1800:200M| 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+---------------+------------+

In [3]:
ol = RFSoCOverlay(path_to_bitstream="./rfsoc_rfdc/bitstream/rfsoc_rfdc_v39_100M.bit")
NEW_CONFIG = {
    "RefClockForPLL": 500.0,
    "DACSampleRate": 4000.0,
    "DACInterpolationRate": 40,
    "DACNCO": 0.0,
    "ADCSampleRate": 2000.0,
    "ADCInterpolationRate": 20,
    "ADCNCO": -0.0,
    "CONFIG_NAME": "rfsoc_rfdc_v39_100M"
}
ZCU216_CONFIG.update(NEW_CONFIG)

adc_datapath/t226/data_mover_ctrl
adc_datapath/t226/fifo_count
axi_gpio_led
dac_datapath/t230/data_mover_ctrl
dac_datapath/t230/fifo_count
usp_rf_data_converter
zynq_ultra_ps_e


In [4]:
import numpy as np

for nco in np.arange(1000.0, 2200.0, 200.0):
    NEW_CONFIG['DACNCO'] = nco
    NEW_CONFIG['ADCNCO'] = -nco
    NEW_CONFIG['CONFIG_NAME'] = "rfsoc_rfdc_v39_100M_" + str(nco)
    print(f"Setting DAC NCO to {nco} and ADC NCO to {-nco}")
    ZCU216_CONFIG.update(NEW_CONFIG)

    rfdc_t = RfdcTask(ol)

    for task in [rfdc_t]:
        task.start()
        task.join()
    
    led_t = BlinkLedTask(ol)
    tx_t = SingleChTxTask(ol)
    rx_t = SingleChRxTask(ol)

    parallel_task = [led_t, tx_t, rx_t]
    for task in parallel_task:
        task.start()

    time.sleep(5)

    for task in parallel_task:
        task.stop()

Setting DAC NCO to 1000.0 and ADC NCO to -1000.0


2024-07-30 21:48:41,679 - root - INFO - Configuring LMK chip using file LMK04828_250.0.txt at 250.0 MHz
2024-07-30 21:48:41,683 - root - INFO - Configuring LMX chip using file LMX2594_500.0.txt at 500.0 MHz
2024-07-30 21:48:44,133 - root - INFO - DAC tile 1 (229) is fully powered up!
2024-07-30 21:48:45,182 - root - INFO - DAC tile 2 (230) is fully powered up!
2024-07-30 21:48:45,185 - root - INFO - DAC tile 0 DAC block 0 is NOT enabled!
2024-07-30 21:48:45,188 - root - INFO - DAC tile 0 DAC block 1 is NOT enabled!
2024-07-30 21:48:45,191 - root - INFO - DAC tile 0 DAC block 2 is NOT enabled!
2024-07-30 21:48:45,194 - root - INFO - DAC tile 0 DAC block 3 is NOT enabled!
2024-07-30 21:48:45,285 - root - INFO - DAC tile 1 DAC block 0 is enabled!
2024-07-30 21:48:45,288 - root - INFO - DAC tile 1 DAC block 1 is NOT enabled!
2024-07-30 21:48:45,291 - root - INFO - DAC tile 1 DAC block 2 is NOT enabled!
2024-07-30 21:48:45,293 - root - INFO - DAC tile 1 DAC block 3 is NOT enabled!
2024-07-3

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'title': {'text': 'Complex Sig…

FigureWidget({
    'data': [{'name': 'Magnitude', 'type': 'scattergl', 'uid': 'f6e26adb-72c1-4db4-ad44-d4d3ed9…

2024-07-30 21:49:05,992 - root - INFO - SNR: 33.518491438894756, CFO: 0.0, EVM: 9.10685224564669, BER: 0.37234375


...

Setting DAC NCO to 2000.0 and ADC NCO to -2000.0


Exception ignored in: <function MyRFdc.__del__ at 0xffff723b6040>
Traceback (most recent call last):
  File "/mnt/sata_ssd_128g/jupyter_notebooks/zcu216_dev/rfsoc_rfdc/rfdc.py", line 208, in __del__
    self.shutdown_tiles()
  File "/mnt/sata_ssd_128g/jupyter_notebooks/zcu216_dev/rfsoc_rfdc/rfdc.py", line 258, in shutdown_tiles
    tile.Shutdown()
AttributeError: 'MyRFdcDACTile' object has no attribute 'Shutdown'
2024-07-30 21:50:28,670 - root - INFO - Configuring LMK chip using file LMK04828_250.0.txt at 250.0 MHz
2024-07-30 21:50:28,674 - root - INFO - Configuring LMX chip using file LMX2594_500.0.txt at 500.0 MHz
2024-07-30 21:50:31,128 - root - INFO - DAC tile 1 (229) is fully powered up!
2024-07-30 21:50:32,177 - root - INFO - DAC tile 2 (230) is fully powered up!
2024-07-30 21:50:32,180 - root - INFO - DAC tile 0 DAC block 0 is NOT enabled!
2024-07-30 21:50:32,183 - root - INFO - DAC tile 0 DAC block 1 is NOT enabled!
2024-07-30 21:50:32,187 - root - INFO - DAC tile 0 DAC block 2

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'title': {'text': 'Complex Sig…

FigureWidget({
    'data': [{'name': 'Magnitude', 'type': 'scattergl', 'uid': '46f23ff2-bdf7-4c4f-b315-f945f29…

2024-07-30 21:50:51,386 - root - INFO - SNR: 18.09707769096423, CFO: 0.0, EVM: 0.7604018111547841, BER: 0.379375
